Import

In [ ]:
!pip install py_midicsv

# Format the data to be midi

In [ ]:
midi_header = '''0, 0, Header, 1, 11, 480
1, 0, Start_track
1, 0, Title_t, ""
1, 0, Time_signature, 4, 2, 24, 8
1, 0, Tempo, 324324
1, 102960, End_track
2, 0, Start_track
2, 0, Control_c, 0, 121, 0
2, 0, Title_t, ""
2, 0, Control_c, 0, 10, 64
2, 0, Control_c, 0, 7, 100
2, 0, Control_c, 0, 11, 127
2, 0, Control_c, 0, 101, 0
2, 0, Control_c, 0, 100, 2
2, 0, Control_c, 0, 6, 64
2, 0, Control_c, 0, 101, 0
2, 0, Control_c, 0, 100, 1
2, 0, Control_c, 0, 6, 64
2, 0, Control_c, 0, 38, 0
2, 0, Control_c, 0, 101, 0
2, 0, Control_c, 0, 100, 0
2, 0, Control_c, 0, 6, 12
2, 0, Pitch_bend_c, 0, 8192
2, 0, Control_c, 0, 1, 0
2, 0, Program_c, 0, 57
'''

Save to CSV

In [ ]:
def generate_on_off_events(noteData):
  track_num = "2"
  #TODO:
  dynamic = 127
  note_on = track_num + ", " + str(noteData[2]) + ", Note_on_c, 0, " + str(noteData[0]) + ", " + str(dynamic)
  note_off = track_num + ", " + str(noteData[2] + noteData[4]) + ", Note_off_c, 0, " + str(noteData[0]) + ", " + "0"
  return [[noteData[2], note_on], [noteData[2] + noteData[4], note_off]]


def test_if_num(num):
  try:
    tmp = int(num)
  except:
    return False

  return True

newCsv = midi_header

file = open("test.txt", "r").read()
fileTokens = file.split("|")
allParsedVals = []

timeMarker = 0
for i in range(len(fileTokens)):
  valuesArr = []
  valueBuild = ""
  for j in range(len(fileTokens[i])):
    if(test_if_num(fileTokens[i][j])):
      valueBuild += fileTokens[i][j]
    else:
      valuesArr.append(int(valueBuild))
      valuesArr.append(fileTokens[i][j])
      valueBuild = ""
  if(valueBuild != ""):
    valuesArr.append(int(valueBuild))
    valueBuild = ""

  oldTime = valuesArr[2]
  valuesArr[2] += timeMarker
  allParsedVals.append(valuesArr)
  timeMarker += oldTime

noteEventsArray = []
for val in allParsedVals:
  events = generate_on_off_events(val)
  noteEventsArray.append(events[0])
  noteEventsArray.append(events[1])
noteEventsArray = sorted(noteEventsArray, key=lambda x: x[0])

outString = midi_header
for val in noteEventsArray:
  outString += val[1] + "\n"
outString += '''2, ''' + str(noteEventsArray[len(noteEventsArray) - 1][0] + 1000) + ''', End_track
0, 0, End_of_file'''

with open('mididata.csv', 'w') as file:
    file.write(outString)

Save as midi

In [ ]:
import py_midicsv as pm

csv_string = "/content/mididata.csv"

# Parse the CSV output of the previous command back into a MIDI file
midi_object = pm.csv_to_midi(csv_string)

# Save the parsed MIDI file to disk
with open("aiOut.mid", "wb") as output_file:
    midi_writer = pm.FileWriter(output_file)
    midi_writer.write(midi_object)